In [ ]:
import pandas as pd



pd.set_option('display.max_columns', 500)

# Load data

df = pd.read_csv('/content/drive/MyDrive/metadata.csv',engine='python', error_bad_lines=False,
                 dtype={'Microsoft Academic Paper ID': str,
                        'pubmed_id': str})

def doi_url(d):
    if d.startswith('http'):
        return d
    elif d.startswith('doi.org'):
        return f'http://{d}'
    else:
        return f'http://doi.org/{d}'
    
df.doi = df.doi.fillna('').apply(doi_url)

print(f'loaded DataFrame with {len(df)} records')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


loaded DataFrame with 482221 records


In [ ]:
len(df)

482221

In [ ]:
df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,http://doi.org/10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,http://doi.org/10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,http://doi.org/10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,http://doi.org/10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,http://doi.org/10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482216,g1u9jods,690a845f253da974c9ce234f607b23b1bc5ee62c,MedRxiv; WHO,An Interpretable Machine Learning Framework fo...,http://doi.org/10.1101/2020.05.18.20105841,NaN,NaN,medrxiv,Effectively and efficiently diagnosing COVID-1...,2020-05-22,"Chen, Y.; Ouyang, L.; Bao, S.; Li, Q.; Han, L....",NaN,NaN,NaN,NaN,document_parses/pdf_json/690a845f253da974c9ce2...,NaN,http://medrxiv.org/cgi/content/short/2020.05.1...,218774271.0
482217,r5ew8ykf,NaN,Elsevier; Medline; PMC,Africa's struggle with inadequate COVID-19 tes...,http://doi.org/10.1016/s2666-5247(20)30014-8,PMC7212979,32835324,cc-by-nc-nd,NaN,2020-05-01,"Adepoju, Paul",The Lancet. Microbe,NaN,NaN,NaN,NaN,NaN,https://api.elsevier.com/content/article/pii/S...,218582881.0
482218,7ri85ri9,NaN,Elsevier; PMC,73 Coronavirus-Specific T-Cells Can be Rapidly...,http://doi.org/10.1016/s2666-6367(21)00099-3,PMC7847408,NaN,els-covid,NaN,2021-03-31,"Keller, Michael D.; Harris, Katherine M.; Conw...",Transplantation and Cellular Therapy,NaN,NaN,NaN,NaN,NaN,https://www.sciencedirect.com/science/article/...,231729758.0
482219,z8piqvf6,7aab206295f8266e21d1a450a585c5e9099bfb76,Medline; PMC,From conception to Covid‐19: An arduous journe...,http://doi.org/10.1111/pbi.13468,PMC7460971,32799416,cc-by,The densely wooded 800‐acre Madras Christian C...,2020-08-16,"Daniell, Henry",Plant Biotechnol J,NaN,NaN,NaN,document_parses/pdf_json/7aab206295f8266e21d1a...,NaN,https://doi.org/10.1111/pbi.13468; https://www...,221143418.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482221 entries, 0 to 482220
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   cord_uid          482221 non-null  object 
 1   sha               160663 non-null  object 
 2   source_x          482221 non-null  object 
 3   title             481987 non-null  object 
 4   doi               482221 non-null  object 
 5   pmcid             170722 non-null  object 
 6   pubmed_id         233427 non-null  object 
 7   license           482221 non-null  object 
 8   abstract          349274 non-null  object 
 9   publish_time      482002 non-null  object 
 10  authors           468641 non-null  object 
 11  journal           450131 non-null  object 
 12  mag_id            0 non-null       float64
 13  who_covidence_id  197114 non-null  object 
 14  arxiv_id          6377 non-null    object 
 15  pdf_json_files    160663 non-null  object 
 16  pmc_json_files    12

In [ ]:
# Helper function for filtering df on abstract + title substring
def abstract_title_filter(search_string):
    return (df.abstract.str.lower().str.replace('-', ' ').str.contains(search_string, na=False) |
            df.title.str.lower().str.replace('-', ' ').str.contains(search_string, na=False))
  

# Helper function which counts synonyms (keywords) and adds tag column to DF
def count_and_tag(df: pd.DataFrame,
                  synonym_list: list,
                  tag_suffix: str) -> (pd.DataFrame, pd.Series):
    counts = {}
    df[f'tag_{tag_suffix}'] = False
    for s in synonym_list:
        synonym_filter = abstract_title_filter(s)
        counts[s] = sum(synonym_filter)
        df.loc[synonym_filter, f'tag_{tag_suffix}'] = True
    return df, pd.Series(counts)

# Function for printing out key passage of abstract based on key terms
def print_key_phrases(df, key_terms, n=5, chars=300):
    for ind, item in enumerate(df[:n].itertuples()):
        print(f'{ind+1} of {len(df)}')
        print(item.title)
        print('[ ' + item.doi + ' ]')
        try:
            i = len(item.abstract)
            for kt in key_terms:
                kt = kt.replace(r'\b', '')
                term_loc = item.abstract.lower().find(kt)
                if term_loc != -1:
                    i = min(i, term_loc)
            if i < len(item.abstract):
                print('    "' + item.abstract[i-30:i+chars-30] + '"')
            else:
                print('    "' + item.abstract[:chars] + '"')
        except:
            print('NO ABSTRACT')
        print('---')

In [ ]:
covid19_synonyms = ['covid',
                    'coronavirus disease 19',
                    'sars cov 2', 
                    '2019 ncov',
                    '2019ncov',
                    r'2019 n cov\b',
                    r'2019n cov\b',
                    'ncov 2019',
                    r'\bn cov 2019',
                    'coronavirus 2019',
                    'wuhan pneumonia',
                    'wuhan virus',
                    'wuhan coronavirus',
                    r'coronavirus 2\b']

df, covid19_counts = count_and_tag(df, covid19_synonyms, 'disease_covid19')

covid19_counts.sort_values(ascending=False)

novel_corona_filter = (abstract_title_filter('novel corona') &
                       df.publish_time.str.startswith('2020', na=False))

# novel corona records published in 2020
print(f'novel corona (published 2020): {sum(novel_corona_filter)}')

# covid-19 tagged records published in 2020
df.loc[novel_corona_filter, 'tag_disease_covid19'] = True
df.tag_disease_covid19.value_counts()

# Confirm all covid-19 records published in 2020 (or missing date)
df[df.tag_disease_covid19].publish_time.str.slice(0, 4).value_counts(dropna=False)

# Mark out earlier papers that are about something else
df.loc[df.tag_disease_covid19 & ~df.publish_time.str.startswith('2020', na=True),
       'tag_disease_covid19'] = False

novel corona (published 2020): 17617


In [ ]:
surv_synonyms = ['surveillance',
                         'syndromic surveillance',
                         'assessment',
                         'testing',
                         'diagnostics',
                         'screening'] 

df, surv_counts = count_and_tag(df, surv_synonyms, 'surv')

surv_counts
    
df.tag_surv.value_counts()  

##crosstab between covid19 and surveillance
df.groupby('tag_disease_covid19').tag_surv.value_counts()

# 1222 papers on Covid-19 and surveillance
n = (df.tag_disease_covid19 & df.tag_surv).sum()
print(f'There are {n} papers on Covid-19 Detection techniques.')

There are 33341 papers on Covid-19 Detection techniques.


In [ ]:
# Abstract excerpt example

# Printing out default 5 examples and key text from  Abstract.

print_key_phrases(df[df.tag_disease_covid19 & df.tag_surv],
                  surv_synonyms)



print_key_phrases(df[df.tag_disease_covid19 & df.tag_surv], surv_synonyms, n=3, chars=100)

1 of 33341
Latest assessment on COVID-19 from the European Centre for Disease Prevention and Control (ECDC)
[ http://doi.org/10.2807/1560-7917.es.2020.25.8.2002271 ]
NO ABSTRACT
---
2 of 33341
Updated rapid risk assessment from ECDC on the outbreak of COVID-19: increased transmission globally
[ http://doi.org/10.2807/1560-7917.es.2020.25.9.2003051 ]
NO ABSTRACT
---
3 of 33341
Updated rapid risk assessment from ECDC on the novel coronavirus disease 2019 (COVID-19) pandemic: increased transmission in the EU/EEA and the UK
[ http://doi.org/10.2807/1560-7917.es.2020.25.10.2003121 ]
NO ABSTRACT
---
4 of 33341
COVID-19 and gender-specific difference: Analysis of public surveillance data in Hong Kong and Shenzhen, China, from January 10 to February 15, 2020
[ http://doi.org/10.1017/ice.2020.64 ]
NO ABSTRACT
---
5 of 33341
Updated rapid risk assessment from ECDC on coronavirus disease 2019 (COVID-19) pandemic: increased transmission in the EU/EEA and the UK
[ http://doi.org/10.2807/1560-7917.e

In [ ]:
bullet1_synonyms =['policy recommendations',
                         'sampling methods',
                         'early detection',]
                         
                         
df, bullet1_counts = count_and_tag(df, bullet1_synonyms, 'bullet1')
bullet1_counts
    
df.tag_bullet1.value_counts()  

# Papers on Covid-19, surveillance, and bullet1
n = (df.tag_disease_covid19 & df.tag_surv & df.tag_bullet1).sum()
print(f'There are {n} papers on Covid-19, early detection and methods:')
print_key_phrases(df[df.tag_disease_covid19 & df.tag_surv & df.tag_bullet1],
                  bullet1_synonyms)

There are 484 papers on Covid-19, early detection and methods:
1 of 484
The Role of Digital Technologies that Could Be Applied for Prescreening in the Mining Industry During the COVID-19 Pandemic
[ http://doi.org/10.1007/s41403-020-00164-0 ]
    "f digital technologies in the early detection and prescreening of emerging infectious and viral diseases to keep mining areas and communities safer and less vulnerable. This paper aims to explore the application of smart digital technologies that could be applied for detection, prescreening and prev"
---
2 of 484
Influenza surveillance system and Covid-19
[ http://doi.org/10.1093/eurpub/ckaa165.354 ]
    "on of ILI SS could succeed in early detection and monitoring of Covid-19 diffusion. We analyzed the data recorded by the influenza SS and we compared the distribution of ILI incidence rate by week for 2017-18, 2018-19 and 2019-20 season in order to understand whether the SS detected any abnormality "
---
3 of 484
Differentiation of COVID-19 c

In [ ]:
prompt = count_and_tag(df, surv_synonyms, 'surv')


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 14.0 MB/s 
     |████████████████████████████████| 895 kB 37.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 48.3 MB/s 
     |████████████████████████████████| 596 kB 37.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
bullet2_synonyms =['existing surveillance',
                   'existing diagnostic']

                         
df, bullet2_counts = count_and_tag(df, bullet2_synonyms, 'bullet2')

bullet2_counts
    
df.tag_bullet2.value_counts()  


# Papers on Covid-19, surveillance, and bullet2
n = (df.tag_disease_covid19 & df.tag_surv & df.tag_bullet2).sum()
print(f'There are {n} papers on Covid-19, existing diagnostic:')
print_key_phrases(df[df.tag_disease_covid19 & df.tag_surv & df.tag_bullet2],
                  bullet2_synonyms)

There are 32 papers on Covid-19, existing diagnostic:
1 of 32
COVID-19 in India: Making a case for the one health surveillance system.
[ http://doi.org/10.4103/ijph.ijph_488_20 ]
    "vides an opinion to boost the existing surveillance activities for early detection and ways to develop an integrated OHS to prevent future COVID-19 like pandemics in India. It also attempts to provide possible solutions at the interface of human-animal-environment, from the simpler to the complex sy"
---
2 of 32
The importance of intra- and inter-institutional networks for capacity building in severe acute respiratory syndrome coronavirus 2 reverse transcription polymerase chain reaction services: experience from an oncology centre in eastern India.
[ http://doi.org/10.4103/ijmm.ijmm_20_255 ]
    " other institutes through the existing diagnostic, surveillance and infection control networks. This networking model for service development and delivery could be used by other centres."
---
3 of 32
Active vacc

In [ ]:
bullet5_synonyms =['rapid test',
                   'rapid tests',
                   'rapid testing',
                   'detetction method',
                   'antigen',
                   'diagnostic']
                   
                         
df, bullet5_counts = count_and_tag(df, bullet5_synonyms, 'bullet5')

bullet5_counts
    
df.tag_bullet5.value_counts()  



n = (df.tag_disease_covid19 & df.tag_surv & df.tag_bullet5).sum()
print(f'There are {n} papers on Covid-19, detection techniques:')
print_key_phrases(df[df.tag_disease_covid19 & df.tag_surv & df.tag_bullet5],
                  bullet5_synonyms)

There are 5343 papers on Covid-19, detection techniques:
1 of 5343
HRCT imaging features in representative imported cases of 2019 novel coronavirus pneumonia
[ http://doi.org/10.1093/pcmedi/pbaa004 ]
    "HRCT) has been one of the key diagnostic tools. To achieve early and accurate diagnostics, determining the radiological characteristics of the disease is of great importance. In this small scale research we retrospectively reviewed and selected six cases confirmed with 2019-nCoV infection in West Ch"
---
2 of 5343
Diagnosis of COVID-19: Considerations, Controversies and Challenges in South Africa
[ http://doi.org/10.18772/26180197.2020.v2nsia1 ]
    "mmunoassays that detect viral antigen in nasopharyngeal samples are being developed and evaluated."
---
3 of 5343
Laboratory Diagnosis of Novel Coronavirus Disease 2019 (COVID-19) Infection
[ http://doi.org/10.1007/978-981-15-4814-7_9 ]
    "st will further intensify the diagnostic modalities of SARS-CoV-2."
---
4 of 5343
SARS-CoV-2-Pande

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import warnings 


warnings.filterwarnings('ignore')

In [ ]:
# load metadata
t1 = time.time()
df =  pd.read_csv('/content/drive/MyDrive/metadata.csv',engine='python', error_bad_lines=False)
t2 = time.time()
print('Elapsed time:', t2-t1)

Elapsed time: 22.98024010658264


In [ ]:
my_keyword = 'rt pcr'

In [ ]:
def word_finder(i_word, i_text):
    found = (str(i_text).lower()).find(str(i_word).lower()) # avoid case sensitivity
    if found == -1:
        result = 0
    else:
        result = 1
    return result

# partial function for mapping
word_indicator_partial = lambda text: word_finder(my_keyword, text)
# build indicator vector (0/1) of hits
keyword_indicator = np.asarray(list(map(word_indicator_partial, df.abstract)))

In [ ]:
print('Number of hits for keyword <', my_keyword, '> : ', keyword_indicator.sum())

Number of hits for keyword < rt pcr > :  107


In [ ]:
import collections
pip install collections

In [ ]:
pip install pyspellchecker

     |████████████████████████████████| 2.7 MB 12.5 MB/s 


In [ ]:
from nltk.corpus import stopwords
#text preprocessing
import re
import string
import pandas as pd

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pip install spellchecker

     |████████████████████████████████| 3.9 MB 11.1 MB/s 
  Created wheel for spellchecker: filename=spellchecker-0.4-py3-none-any.whl size=3966515 sha256=c42b9352da989e2c90212fc8a49721a64df924d9f50f85bed5fb63e67f1e04f5
  Stored in directory: /root/.cache/pip/wheels/58/e9/48/b82b733a7a0a9cc52ed239ccf082ff33e2fbda71670ddd3349
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7141 sha256=a1946e8c06eebf02dc969558f8c42d6e8b220d44e5b556678ca2f2cb938707ac
  Stored in directory: /root/.cache/pip/wheels/3c/f3/5d/77e8983e86af1c3c62b80d76de8020f560ce6367f1b2763b40
  Created wheel for silpa-common: filename=silpa_common-0.3-py3-none-any.whl size=8483 sha256=ce9f4767b8e1cc3a9e5a8d6731f83f16ea03ac72719f9a2bf0e2d8c157e25e66
  Stored in directory: /root/.cache/pip/wheels/02/81/11/40ef1cf40f5c4021a214653ca3362914f0f9e14e8322f75f9b
  Created wheel for soundex: filename=soundex-1.1.3-py3-none-any.whl size=8894 sha256=97a644e8860e4d413fbbe69cae1c4ad9deed565704da8567a44

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install nltk

In [ ]:
df['abstract']

0         OBJECTIVE: This retrospective chart review des...
1         Inflammatory diseases of the respiratory tract...
2         Surfactant protein-D (SP-D) participates in th...
3         Endothelin-1 (ET-1) is a 21 amino acid peptide...
4         Respiratory syncytial virus (RSV) and pneumoni...
                                ...                        
482216    Effectively and efficiently diagnosing COVID-1...
482217                                                  NaN
482218                                                  NaN
482219    The densely wooded 800‐acre Madras Christian C...
482220    OBJECTIVES: From 2016 to 2018 Florida document...
Name: abstract, Length: 482221, dtype: object

In [ ]:
df['abstract']=df['abstract'].str.lower()
type(df['abstract'])

pandas.core.series.Series

In [ ]:
df_new = df[df['abstract'].notnull()]

In [ ]:
#stopwords removal
import re
import string
eng_stopwords = set(stopwords.words('english'))

print('Some of the english stopwords: ', list(eng_stopwords)[:10])
df_new['stopwords'] = df_new['abstract'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in eng_stopwords]))
df_new[['abstract', 'stopwords']].head()


Some of the english stopwords:  ['does', 'ours', 'themselves', 't', 'll', 'they', 'by', 've', 'wouldn', 'will']


,abstract,stopwords
0,objective: this retrospective chart review des...,objective: retrospective chart review describe...
1,inflammatory diseases of the respiratory tract...,inflammatory diseases respiratory tract common...
2,surfactant protein-d (sp-d) participates in th...,surfactant protein-d (sp-d) participates innat...
3,endothelin-1 (et-1) is a 21 amino acid peptide...,endothelin-1 (et-1) 21 amino acid peptide dive...
4,respiratory syncytial virus (rsv) and pneumoni...,respiratory syncytial virus (rsv) pneumonia vi...


In [ ]:

#3. Removing URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text).replace('  ', ' ')

df_new['url'] = df_new['stopwords'].apply(lambda x: remove_urls(x))
df_new[['abstract', 'url']].head()

#4. Punctuation Removal
punctuation = string.punctuation

#print(punctuation)

def remove_punctuation(text):
    text = text.translate(str.maketrans('', '', punctuation))
    return text
  
df_new['punctuation'] = df_new['stopwords'].apply(lambda x: remove_punctuation(x))
df_new[['abstract', 'punctuation']].head()

,abstract,punctuation
0,objective: this retrospective chart review des...,objective retrospective chart review describes...
1,inflammatory diseases of the respiratory tract...,inflammatory diseases respiratory tract common...
2,surfactant protein-d (sp-d) participates in th...,surfactant proteind spd participates innate re...
3,endothelin-1 (et-1) is a 21 amino acid peptide...,endothelin1 et1 21 amino acid peptide diverse ...
4,respiratory syncytial virus (rsv) and pneumoni...,respiratory syncytial virus rsv pneumonia viru...


In [ ]:
ch =df_new['punctuation'][1]
lis=ch.split()
lis=list(lis)

from collections import Counter
count=Counter(lis)
most_occur=count.most_common(8)
most_occur

[('no•', 7),
 ('evidence', 3),
 ('inflammatory', 2),
 ('diseases', 2),
 ('respiratory', 2),
 ('tract', 2),
 ('dependent', 2),
 ('although', 2)]

In [ ]:
#5. Stemming
stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in word_tokenize(text)])

df_new['text_stemmed'] = df_new['punctuation'].apply(lambda x: stem_words(x))
df_new[['abstract', 'text_stemmed']].head()

,abstract,text_stemmed
0,objective: this retrospective chart review des...,object retrospect chart review describ epidemi...
1,inflammatory diseases of the respiratory tract...,inflammatori diseas respiratori tract commonli...
2,surfactant protein-d (sp-d) participates in th...,surfact proteind spd particip innat respons in...
3,endothelin-1 (et-1) is a 21 amino acid peptide...,endothelin1 et1 21 amino acid peptid divers bi...
4,respiratory syncytial virus (rsv) and pneumoni...,respiratori syncyti viru rsv pneumonia viru mi...


In [ ]:
pip install PyDictionary

  Created wheel for goslate: filename=goslate-1.5.2-py3-none-any.whl size=11436 sha256=ace7ccbf0e0c95d2ebda07fa159933299702b4fcb367d50f33e8251d4f345755
  Stored in directory: /root/.cache/pip/wheels/a8/8a/c4/85425eac5e0746fd5fc898801858331e55ac386f476d65e58d
Successfully built goslate


In [ ]:
para=df_new['text_stemmed'][1]
para

'inflammatori diseas respiratori tract commonli associ elev product nitric oxid no• increas indic no• depend oxid stress although no• known antimicrobi antiinflammatori antioxid properti variou line evid support contribut no• lung injuri sever diseas model basi biochem evid often presum no• depend oxid due format oxid peroxynitrit although altern mechan involv phagocytederiv heme protein myeloperoxidas eosinophil peroxidas might oper condit inflamm overwhelm literatur no• gener activ respiratori tract would beyond scope commentari review area comprehens instead focus recent evid concept presum contribut no• inflammatori diseas lung'

In [ ]:
para1=para.split()
para1

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus.reader.wordnet import Synset
from nltk.corpus import wordnet
s=set()
for i in para1:
  for j in wordnet.synsets(i):
    #print(i,j)
    for l in j.lemmas():
      s.add(i)
      
print(s)

{'recent', 'format', 'tract', 'beyond', 'model', 'due', 'review', 'depend', 'nitric', 'lung', 'indic', 'product', 'might', 'heme', 'eosinophil', 'overwhelm', 'focus', 'area', 'known', 'stress', 'support', 'sever', 'protein', 'often', 'concept', 'instead', 'scope', 'line'}


In [ ]:
d=set(para1)

In [ ]:
non_syno=d-s

In [ ]:
print(non_syno)

{'would', 'condit', 'inflamm', 'phagocytederiv', 'associ', 'antioxid', 'although', 'comprehens', 'contribut', 'presum', 'mechan', 'gener', 'myeloperoxidas', 'antiinflammatori', 'increas', 'oper', 'no•', 'biochem', 'involv', 'injuri', 'evid', 'diseas', 'antimicrobi', 'activ', 'basi', 'inflammatori', 'altern', 'commentari', 'peroxidas', 'respiratori', 'peroxynitrit', 'elev', 'oxid', 'commonli', 'properti', 'literatur', 'variou'}


In [ ]:
ch =df_new['text_stemmed'][1]
lis=ch.split()
lis=list(lis)

from collections import Counter
count=Counter(lis)
most_occur=count.most_common(4)
most_occur

[('no•', 7), ('oxid', 4), ('diseas', 3), ('evid', 3)]

In [ ]:
lis

In [ ]:
#6. Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df_new["text_lemmatized"] = df_new["punctuation"].apply(lambda text: lemmatize_words(text))
df_new[['abstract', 'text_lemmatized']].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,abstract,text_lemmatized
0,objective: this retrospective chart review des...,objective retrospective chart review describes...
1,inflammatory diseases of the respiratory tract...,inflammatory disease respiratory tract commonl...
2,surfactant protein-d (sp-d) participates in th...,surfactant proteind spd participates innate re...
3,endothelin-1 (et-1) is a 21 amino acid peptide...,endothelin1 et1 21 amino acid peptide diverse ...
4,respiratory syncytial virus (rsv) and pneumoni...,respiratory syncytial virus rsv pneumonia viru...


In [ ]:
para_lem=df_new['text_lemmatized'][1]
para_lem

'inflammatory disease respiratory tract commonly associated elevated production nitric oxide no• increased index no• dependent oxidative stress although no• known antimicrobial antiinflammatory antioxidant property various line evidence support contribution no• lung injury several disease model basis biochemical evidence often presumed no• dependent oxidation due formation oxidant peroxynitrite although alternative mechanism involving phagocytederived heme protein myeloperoxidase eosinophil peroxidase might operative condition inflammation overwhelming literature no• generation activity respiratory tract would beyond scope commentary review area comprehensively instead focus recent evidence concept presumed contribution no• inflammatory disease lung'

In [ ]:
para_lem=para.split()
para_lem

['inflammatori',
 'diseas',
 'respiratori',
 'tract',
 'commonli',
 'associ',
 'elev',
 'product',
 'nitric',
 'oxid',
 'no•',
 'increas',
 'indic',
 'no•',
 'depend',
 'oxid',
 'stress',
 'although',
 'no•',
 'known',
 'antimicrobi',
 'antiinflammatori',
 'antioxid',
 'properti',
 'variou',
 'line',
 'evid',
 'support',
 'contribut',
 'no•',
 'lung',
 'injuri',
 'sever',
 'diseas',
 'model',
 'basi',
 'biochem',
 'evid',
 'often',
 'presum',
 'no•',
 'depend',
 'oxid',
 'due',
 'format',
 'oxid',
 'peroxynitrit',
 'although',
 'altern',
 'mechan',
 'involv',
 'phagocytederiv',
 'heme',
 'protein',
 'myeloperoxidas',
 'eosinophil',
 'peroxidas',
 'might',
 'oper',
 'condit',
 'inflamm',
 'overwhelm',
 'literatur',
 'no•',
 'gener',
 'activ',
 'respiratori',
 'tract',
 'would',
 'beyond',
 'scope',
 'commentari',
 'review',
 'area',
 'comprehens',
 'instead',
 'focus',
 'recent',
 'evid',
 'concept',
 'presum',
 'contribut',
 'no•',
 'inflammatori',
 'diseas',
 'lung']

In [ ]:
from nltk.corpus.reader.wordnet import Synset
from nltk.corpus import wordnet
s=set()
for i in para_lem:
  for j in wordnet.synsets(i):
    #print(i,j)
    for l in j.lemmas():
      s.add(i)
      
print(s)

{'recent', 'format', 'tract', 'beyond', 'model', 'due', 'review', 'depend', 'nitric', 'lung', 'indic', 'product', 'might', 'heme', 'eosinophil', 'overwhelm', 'focus', 'area', 'known', 'stress', 'support', 'sever', 'protein', 'often', 'concept', 'instead', 'scope', 'line'}


In [ ]:
d=set(para_lem)

In [ ]:
diff=d-s
print(diff)

{'would', 'condit', 'inflamm', 'phagocytederiv', 'associ', 'antioxid', 'although', 'comprehens', 'contribut', 'presum', 'mechan', 'gener', 'myeloperoxidas', 'antiinflammatori', 'increas', 'oper', 'no•', 'biochem', 'involv', 'injuri', 'evid', 'diseas', 'antimicrobi', 'activ', 'basi', 'inflammatori', 'altern', 'commentari', 'peroxidas', 'respiratori', 'peroxynitrit', 'elev', 'oxid', 'commonli', 'properti', 'literatur', 'variou'}


In [ ]:
ch =df_new['text_lemmatized'][1]
lis=ch.split()
lis=list(lis)

from collections import Counter
count=Counter(lis)
most_occur=count.most_common(4)
most_occur


[('of', 13), ('the', 11), ('no•', 6), ('and', 5)]

In [ ]:
df['selection'] = keyword_indicator

# select only hits from data frame
df_hits = df[df['selection']==1]
# show results
df_hits

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,selection
11058,gdpxn37e,0654274bb77dd4ca5973bad0cbe0050f97621d62,PMC,An increasing prevalence of non-GII.4 noroviru...,10.1007/s00705-020-04599-2,PMC7222896,32221714,no-cc,"Since 2014, novel non-GII.4 norovirus (NoV) ge...",2020-03-27,"Chen, Liping; Xu, Deshun; Wu, Xiaofang; Liu, G...",Arch Virol,NaN,NaN,NaN,document_parses/pdf_json/0654274bb77dd4ca5973b...,document_parses/pmc_json/PMC7222896.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,1
15560,a9wi78ur,5401fd9fb66aa36da36c7b576ded41be61daa1e1,PMC,A double blinded placebo controlled comparativ...,10.1186/s13063-021-05041-x,PMC7876527,33573696,cc-by,OBJECTIVES: The primary objectives of the stud...,2021-02-11,"Srivastava, Anurag; Rengaraju, Manickavasagam;...",Trials,NaN,NaN,NaN,document_parses/pdf_json/5401fd9fb66aa36da36c7...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,1
30436,v8mcvnlw,08aef0b467a8adb264af4ad4820662a707118787,PMC,An Overview of Coronavirus COVID-19 with their...,10.1007/s41403-020-00147-1,PMC7336099,NaN,no-cc,"COVID-19 has created a devastating pandemic, i...",2020-07-06,"Biswas, Tapan Krishna; Biswas, Malabika; Bandy...",Trans Indian Natl,NaN,NaN,NaN,document_parses/pdf_json/08aef0b467a8adb264af4...,document_parses/pmc_json/PMC7336099.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,1
45836,fgm91ixh,NaN,Medline,The use of routine blood tests to assist the d...,10.1177/0004563221999076,NaN,33591793,unk,IntroductionSpecific patterns of blood test re...,2021-02-16,"Parsons, Iain Thomas; Parsons, Alan T; Balme, ...",Annals of clinical biochemistry,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1177/0004563221999076; http...,231944694.0,1
60238,mtw5eqyu,NaN,Medline,COVID-19 S: A new proposal for diagnosis and s...,10.5152/dir.2020.20351,NaN,32558646,unk,PURPOSE Because of the widespread use of CT in...,2020-06-19,"Gezer, Naciye Sinem; Ergan, Begüm; Barış, Must...",Diagnostic and interventional radiology,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.5152/dir.2020.20351; https:...,219911103.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464056,gkxesak9,7573052026e55ad7d4b410171fc36b532268d185,Medline; PMC,Pediatric E.N.T. emergencies during COVID-19 p...,10.1007/s12070-020-02357-z,PMC7781556,33425696,no-cc,Our aim is to analyze effect of covid-19 pande...,2021-01-04,"Palas, Anand; Raval, Jayman; Aiyer, R. G.; Aru...",Indian J Otolaryngol Head Neck Surg,NaN,NaN,NaN,document_parses/pdf_json/7573052026e55ad7d4b41...,document_parses/pmc_json/PMC7781556.xml.json,https://doi.org/10.1007/s12070-020-02357-z; ht...,230610344.0,1
476748,6rqi17oz,20fa9e9ac66e7149b804f6984ad7216487f4adc2,Medline; PMC,Sperm cryopreservation during the SARS-CoV-2 p...,10.1007/s40618-020-01438-8,PMC7547554,33040303,cc-by,PURPOSE: Sperm cryopreservation is fundamental...,2020-10-10,"Paoli, D.; Pallotti, F.; Nigro, G.; Aureli, A....",J Endocrinol Invest,NaN,NaN,NaN,document_parses/pdf_json/20fa9e9ac66e7149b804f...,document_parses/pmc_json/PMC7547554.xml.json,https://doi.org/10.1007/s40618-020-01438-8; ht...,222254113.0,1
479070,2uz3sjg6,de68fb2ade53427ea7561383b801b3082e0a0126,MedRxiv; WHO,Chyawanprash for the prevention of COVID-19 in...,10.1101/2021.02.17.21251899,NaN,NaN,medrxiv,Background: Coronavirus disease 2019 (Covid19)...,2021-02-19,"Gupta, A.; Madan, A.; Yadav, B.; Mundada, P. S...",NaN,NaN,NaN,NaN,document_parses/pdf_json/de68fb2ade53427ea7561...,NaN,http://medrxiv.org/cgi/content/short/2021.02.1...,231960010.0,1
481354,2uyaycwz,a2373788b3199f709c0686dec692ec9030a2ac0c,Medline; PMC,Epidemiological study of influenza virus infec...,10.1111/j.1750-2659.2008.00048.x,PMC4634226,19453464,no-cc,Background Influenza virus is the most common ...,2008-08-03,"Santamaría, Cecilia; Urueña, Analía; Videla, C...",Influenza Other Respir Viruses,NaN,NaN,NaN,document_

In [ ]:
df_hits.iloc[15]

cord_uid                                                     in6w5d2y
sha                          6338a6003446f87e273e5e6faea242b49e094891
source_x                                                          PMC
title               Simian virus 40 vectors for pulmonary gene the...
doi                                            10.1186/1465-9921-8-74
pmcid                                                      PMC2238754
pubmed_id                                                    17967178
license                                                         cc-by
abstract            BACKGROUND: Sepsis remains the leading cause o...
publish_time                                               2007-10-29
authors             Eid, Luminita; Bromberg, Zohar; EL-Latif, Mahm...
journal                                                    Respir Res
mag_id                                                            NaN
who_covidence_id                                                  NaN
arxiv_id            

In [ ]:
df_hits.abstract.iloc[1:5]

15560    OBJECTIVES: The primary objectives of the stud...
30436    COVID-19 has created a devastating pandemic, i...
45836    IntroductionSpecific patterns of blood test re...
60238    PURPOSE Because of the widespread use of CT in...
Name: abstract, dtype: object

In [ ]:
df_hits.abstract.iloc[24]

"The increasing COVID-19 widespread has created the necessity to assess the diagnostic accuracy of newly introduced (RT-PCR based) assays for SARS-CoV-2 RNA detection in respiratory tract samples. We compared the results of the Allplex&#8482; 2019-nCoV assay with those of the Simplexa&#8482; COVID-19 Direct assay and the Quanty COVID-19 assay, respectively, all performed on 125 nasal/oropharyngeal swab samples of patients with COVID-19 suspicion. Fifty-four samples were positive, and 71 were negative with the Allplex&#8482; assay, whereas 47 of 54 samples were also positive with the Simplexa&#8482; assay. The Quanty assay detected 55 positive samples, including the 54 positive samples with the Allplex&#8482; assay and 1 sample that was Allplex&#8482; negative but Simplexa&#8482; positive. Using a consensus result criterion as the reference standard allowed to resolve the eight samples with discordant results (one Allplex&#8482; negative and seven Simplexa&#8482; negative) as truly fals

In [ ]:
my_keyword2 = 'diagnostic'

In [ ]:
word_indicator_partial2 = lambda text: word_finder(my_keyword2, text)
# build indicator vector (0/1) of hits
keyword_indicator2 = np.asarray(list(map(word_indicator_partial2, df_hits.abstract)))

In [ ]:
print('Number of hits for keyword <', my_keyword2, '> : ', keyword_indicator.sum())

Number of hits for keyword < diagnostic > :  20826


In [ ]:
df_hits['selection'] = keyword_indicator2

# select only hits from data frame
df_his2 = df_hits[df_hits['selection']==1]
df_his2

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,selection
15,t35n7bk9,30a4842a2e257f725cc041e32fd682c495111a50,PMC,"Multi-faceted, multi-versatile microarray: sim...",10.1186/1742-4690-1-11,PMC420498,15169556,no-cc,There are hundreds of viruses that infect diff...,2004-05-26,"Shieh, Biehuoy; Li, Ching",Retrovirology,NaN,NaN,NaN,document_parses/pdf_json/30a4842a2e257f725cc04...,document_parses/pmc_json/PMC420498.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN,1
346,fmgnavfq,d0c33264c3dc13c6c5f3a3d83bdde246bdc82165,PMC,Clinical Assessment and Improved Diagnosis of ...,10.3201/eid1509.090204,PMC2819894,19788810,no-cc,Human bocavirus (HBoV) is a widespread respira...,2009-09-08,"Söderlund-Venermo, Maria; Lahtinen, Anne; Jart...",Emerg Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d0c33264c3dc13c6c5f3a...,document_parses/pmc_json/PMC2819894.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,1
499,8ctsa9sd,49ef14fbeee682057ed14674f623ca36c6d954cc,PMC,RNA and DNA Bacteriophages as Molecular Diagno...,10.1371/journal.pone.0016142,PMC3036576,21347398,cc-by,Real-time PCR techniques are now commonly used...,2011-02-09,"Ninove, Laetitia; Nougairede, Antoine; Gazin, ...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/49ef14fbeee682057ed14...,document_parses/pmc_json/PMC3036576.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,1
554,qol5poan,133933d03341e293921239f0d6a2b3ea4f1f574c,PMC,"A Simple, Inexpensive Device for Nucleic Acid ...",10.1371/journal.pone.0019738,PMC3090398,21573065,cc-by,BACKGROUND: Molecular assays targeted to nucle...,2011-05-09,"LaBarre, Paul; Hawkins, Kenneth R.; Gerlach, J...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/133933d03341e29392123...,document_parses/pmc_json/PMC3090398.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,1
591,qjmyc2tu,d7011047894121dc7f115cbeb1f355b76c33f80d,PMC,A Brief Review on Diagnosis of Foot-and-Mouth ...,10.4061/2011/905768,PMC3135314,21776357,no-cc,Foot-and-mouth disease (FMD) is one of the hig...,2011-07-06,"Longjam, Neeta; Deb, Rajib; Sarmah, A. K.; Tay...",Vet Med Int,NaN,NaN,NaN,document_parses/pdf_json/d7011047894121dc7f115...,document_parses/pmc_json/PMC3135314.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481951,6tg5pr4a,4b20494e5659c33672ba06eb82c2cf6eaa8d7566,Medline; PMC,Evaluation of predictive value of olfactory dy...,10.1002/lio2.482,PMC7752039,33364386,cc-by-nc-nd,"OBJECTIVES: At the end of 2019, SARS‐CoV‐2 was...",2020-10-22,"Romero‐Gameros, Carlos Alfonso; Waizel‐Haiat, ...",Laryngoscope Investig Otolaryngol,NaN,NaN,NaN,document_parses/pdf_json/4b20494e5659c33672ba0...,document_parses/pmc_json/PMC7752039.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33364386/;...,226336685.0,1
481973,6v4hb0s6,42a0137eef00cdb0ffc719a5937be65ba0a5d392,Medline; PMC,Evaluation of SARS-CoV-2 antigen-based rapid d...,10.1186/s12985-021-01505-3,PMC7881305,33581714,cc-by,Rapid diagnosis of SARS-CoV-2 during pandemic ...,2021-02-13,"Saeed, Umar; Uppal, Sara Rizwan; Piracha, Zahr...",Virol J,NaN,NaN,NaN,document_parses/pdf_json/42a0137eef00cdb0ffc71...,document_parses/pmc_json/PMC7881305.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33581714/;...,231913405.0,1
482028,gi6v12mi,bb43dcd3e23079b8695f69073ced2e9da93514b8,Elsevier; Medline; PMC,End-point RT-PCR: a potential alternative for ...,10.1016/j.jviromet.2020.114007,PMC7598561,33130151,no-cc,Real-time reverse transcription-polymerase cha...,2020-10-29,"Silva Júnior, José Valter Joaquim; Merchioratt...",J Virol Methods,NaN,NaN,NaN,document_parses/pdf_json/bb43dcd3e23079b8695f6...,document_parses/pmc_json/PMC7598561.xml.json,https://www.sciencedirect.com/science/article/...,226199849.0,1
482050,uaw0j7gq,858025afd11f8bfd856ca988c7587d0356982966,MedRxiv; WHO,Characterising 

In [ ]:
df_his2.iloc[0]

cord_uid                                                     t35n7bk9
sha                          30a4842a2e257f725cc041e32fd682c495111a50
source_x                                                          PMC
title               Multi-faceted, multi-versatile microarray: sim...
doi                                            10.1186/1742-4690-1-11
pmcid                                                       PMC420498
pubmed_id                                                    15169556
license                                                         no-cc
abstract            There are hundreds of viruses that infect diff...
publish_time                                               2004-05-26
authors                                     Shieh, Biehuoy; Li, Ching
journal                                                 Retrovirology
mag_id                                                            NaN
who_covidence_id                                                  NaN
arxiv_id            

In [ ]:
df_his2.abstract.iloc[0]

'There are hundreds of viruses that infect different human organs and cause diseases. Some fatal emerging viral infections have become serious public health issues worldwide. Early diagnosis and subsequent treatment are therefore essential for fighting viral infections. Current diagnostic techniques frequently employ polymerase chain reaction (PCR)-based methods to quickly detect the pathogenic viruses and establish the etiology of the disease or illness. However, the fast PCR method suffers from many drawbacks such as a high false-positive rate and the ability to detect only one or a few gene targets at a time. Microarray technology solves the problems of the PCR limitations and can be effectively applied to all fields of molecular medicine. Recently, a report in Retrovirology described a multi-virus DNA array that contains more than 250 open reading frames from eight human viruses including human immunodeficiency virus type 1. This array can be used to detect multiple viral co-infect

In [ ]:
my_keyword3 = 'detection'

In [ ]:
word_indicator_partial2 = lambda text: word_finder(my_keyword3, text)
# build indicator vector (0/1) of hits
keyword_indicator3 = np.asarray(list(map(word_indicator_partial2, df_hits.abstract)))

In [ ]:
print('Number of hits for keywords <', my_keyword, '> + <', my_keyword2, '> + <', my_keyword3, '> : ', keyword_indicator3.sum())

Number of hits for keywords < pcr > + < diagnostic > + < detection > :  5931


In [ ]:
def word_finder(i_word, i_text):
    found = (str(i_text).lower()).find(str(i_word).lower()) # avoid case sensitivity
    if found == -1:
        result = 0
    else:
        result = 1
    return result

In [ ]:
# partial function for mapping
word_indicator_partial = lambda text: word_finder(my_keyword, text)
# build indicator vector (0/1) of hits
keyword_indicator = np.asarray(list(map(word_indicator_partial, df.abstract)))

In [ ]:
df_hits['selection'] = keyword_indicator3

# select only hits from data frame
df_hi = df_hits[df_hits['selection']==1]
df_hi

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,selection
29,1pq6dkl5,184aded923f0ac3cbdbcf74d2a5b42cda0f414c2,PMC,Towards standardization of RNA quality assessm...,10.1093/nar/gni054,PMC1072807,15800207,no-cc,While it is universally accepted that intact R...,2005-03-30,"Imbeaud, Sandrine; Graudens, Esther; Boulanger...",Nucleic Acids Res,NaN,NaN,NaN,document_parses/pdf_json/184aded923f0ac3cbdbcf...,document_parses/pmc_json/PMC1072807.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,1
41,qva0jt86,4ba79e54ecf81b30b56461a6aec2094eaf7b7f06,PMC,Relevance of human metapneumovirus in exacerba...,10.1186/1465-9921-6-150,PMC1334186,16371156,cc-by,BACKGROUND AND METHODS: Human metapneumovirus ...,2005-12-21,"Rohde, G; Borg, I; Arinir, U; Kronsbein, J; Ra...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/4ba79e54ecf81b30b5646...,document_parses/pmc_json/PMC1334186.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,1
66,p5jtwb3l,NaN,PMC,Electrochemical Molecular Analysis Without Nuc...,10.1016/j.ymeth.2005.05.008,PMC1564062,16213156,green-oa,Electrochemical biosensors have revolutionized...,2005-09-01,"Gau, Vincent; Ma, Shu-Ching; Wang, Hua; Tsukud...",Methods,NaN,NaN,NaN,NaN,document_parses/pmc_json/PMC1564062.xml.json,http://europepmc.org/articles/pmc1564062?pdf=r...,NaN,1
77,kt9pmfdr,8b39433dd865c0f71c7b2f333e1f506b73d722f1; 5256...,PMC,A novel endonuclease IV post-PCR genotyping sy...,10.1093/nar/gkl679,PMC1636472,17012270,cc-by-nc,Here we describe a novel endonuclease IV (Endo...,2006-09-29,"Kutyavin, Igor V.; Milesi, Dave; Belousov, Yev...",Nucleic Acids Res,NaN,NaN,NaN,document_parses/pdf_json/8b39433dd865c0f71c7b2...,document_parses/pmc_json/PMC1636472.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,1
132,in6w5d2y,6338a6003446f87e273e5e6faea242b49e094891,PMC,Simian virus 40 vectors for pulmonary gene the...,10.1186/1465-9921-8-74,PMC2238754,17967178,cc-by,BACKGROUND: Sepsis remains the leading cause o...,2007-10-29,"Eid, Luminita; Bromberg, Zohar; EL-Latif, Mahm...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6338a6003446f87e273e5...,document_parses/pmc_json/PMC2238754.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481973,6v4hb0s6,42a0137eef00cdb0ffc719a5937be65ba0a5d392,Medline; PMC,Evaluation of SARS-CoV-2 antigen-based rapid d...,10.1186/s12985-021-01505-3,PMC7881305,33581714,cc-by,Rapid diagnosis of SARS-CoV-2 during pandemic ...,2021-02-13,"Saeed, Umar; Uppal, Sara Rizwan; Piracha, Zahr...",Virol J,NaN,NaN,NaN,document_parses/pdf_json/42a0137eef00cdb0ffc71...,document_parses/pmc_json/PMC7881305.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33581714/;...,231913405.0,1
482028,gi6v12mi,bb43dcd3e23079b8695f69073ced2e9da93514b8,Elsevier; Medline; PMC,End-point RT-PCR: a potential alternative for ...,10.1016/j.jviromet.2020.114007,PMC7598561,33130151,no-cc,Real-time reverse transcription-polymerase cha...,2020-10-29,"Silva Júnior, José Valter Joaquim; Merchioratt...",J Virol Methods,NaN,NaN,NaN,document_parses/pdf_json/bb43dcd3e23079b8695f6...,document_parses/pmc_json/PMC7598561.xml.json,https://www.sciencedirect.com/science/article/...,226199849.0,1
482053,58jlfaa4,00a46273b4ba1435d6bdae3e8a3bfb20011f1ef7,BioRxiv; WHO,Thermodynamic evaluation of the impact of DNA ...,10.1101/2020.11.04.368449,NaN,NaN,biorxiv,Background DNA mismatches can affect the effic...,2020-11-05,"Miranda, Pâmella; Weber, Gerald",bioRxiv,NaN,NaN,NaN,document_parses/pdf_json/00a46273b4ba1435d6bda...,NaN,https://doi.org/10.1101/2020.11.04.368449,226959343.0,1
482062,6mdimxnk,7a6965ea1164ac5ad73e0f499659d413861b9861,MedRxiv; WHO,Analytical sensitivity and efficiency comparis...,10.1101/2020.03.30.20048108,NaN,NaN,medrxiv,The recent spread of severe acute respiratory ...,2020-04-01,"Vogels, Chantal 

In [ ]:
df_hi.abstract.iloc[0]

'While it is universally accepted that intact RNA constitutes the best representation of the steady-state of transcription, there is no gold standard to define RNA quality prior to gene expression analysis. In this report, we evaluated the reliability of conventional methods for RNA quality assessment including UV spectroscopy and 28S:18S area ratios, and demonstrated their inconsistency. We then used two new freely available classifiers, the Degradometer and RIN systems, to produce user-independent RNA quality metrics, based on analysis of microcapillary electrophoresis traces. Both provided highly informative and valuable data and the results were found highly correlated, while the RIN system gave more reliable data. The relevance of the RNA quality metrics for assessment of gene expression differences was tested by Q-PCR, revealing a significant decline of the relative expression of genes in RNA samples of disparate quality, while samples of similar, even poor integrity were found h

In [ ]:
my_keyword4 = 'positive'

In [ ]:
word_indicator_partial2 = lambda text: word_finder(my_keyword4, text)
# build indicator vector (0/1) of hits
keyword_indicator4 = np.asarray(list(map(word_indicator_partial2, df_hits.abstract)))

In [ ]:
print('Number of hits for keywords <', my_keyword, '> + <', my_keyword2, '> + <', my_keyword3,'> + <', my_keyword4, '> : ', keyword_indicator4.sum())

Number of hits for keywords < pcr > + < diagnostic > + < detection > + < positive > :  9917


In [ ]:
def word_finder(i_word, i_text):
    found = (str(i_text).lower()).find(str(i_word).lower()) # avoid case sensitivity
    if found == -1:
        result = 0
    else:
        result = 1
    return result

In [ ]:
# partial function for mapping
word_indicator_partial = lambda text: word_finder(my_keyword4, text)
# build indicator vector (0/1) of hits
keyword_indicator = np.asarray(list(map(word_indicator_partial, df.abstract)))

In [ ]:
my_keyword5 = 'negative'

In [ ]:
word_indicator_partial2 = lambda text: word_finder(my_keyword5, text)
# build indicator vector (0/1) of hits
keyword_indicator5 = np.asarray(list(map(word_indicator_partial2, df_hits.abstract)))

In [ ]:
print('Number of hits for keywords <', my_keyword, '> + <', my_keyword2, '> + <', my_keyword3,'> + <', my_keyword4,'> + <', my_keyword5, '> : ', keyword_indicator5.sum())

Number of hits for keywords < pcr > + < diagnostic > + < detection > + < positive > + < negative > :  5511


In [ ]:
def word_finder(i_word, i_text):
    found = (str(i_text).lower()).find(str(i_word).lower()) # avoid case sensitivity
    if found == -1:
        result = 0
    else:
        result = 1
    return result

In [ ]:
# partial function for mapping
word_indicator_partial = lambda text: word_finder(my_keyword5, text)
# build indicator vector (0/1) of hits
keyword_indicator5 = np.asarray(list(map(word_indicator_partial, df.abstract)))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import glob
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from  collections import OrderedDict


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/metadata.csv")
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [ ]:
!pip install bert-extractive-summarizer

     |████████████████████████████████| 3.8 MB 7.6 MB/s 
     |████████████████████████████████| 67 kB 2.2 MB/s 
     |████████████████████████████████| 895 kB 36.1 MB/s 
     |████████████████████████████████| 596 kB 38.4 MB/s 
     |████████████████████████████████| 6.5 MB 26.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#BERT
body=df['abstract'][0]
from summarizer import Summarizer
model = Summarizer()
result = model(body, min_length=50,max_length=100)
full0 = ''.join(result)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(full0)

RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most patients with pneumonia had crepitations (79.2%) but only 25% had bronchial breathing.


In [ ]:
#GPT2
body=df['abstract'][0]
from summarizer import Summarizer,TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(body, min_length=50, max_length=100))

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
print(full)

RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. More than three-quarters of patients (77.5%) had comorbidities.


In [ ]:
% pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.7 MB/s 


In [ ]:
keyword_indicator5 = np.asarray(list(map(word_indicator_partial, df.abstract)))

In [ ]:
!pip install rouge_score

In [ ]:
!pip install transformers

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rouge3','rougeL'])
scorer.score(df_hits.abstract.iloc[1],df_hits.abstract.iloc[2])

{'rouge1': Score(precision=0.48427672955974843, recall=0.09821428571428571, fmeasure=0.1633085896076352),
 'rouge2': Score(precision=0.11392405063291139, recall=0.022988505747126436, fmeasure=0.03825717321997875),
 'rouge3': Score(precision=0.025477707006369428, recall=0.005115089514066497, fmeasure=0.008519701810436636),
 'rougeL': Score(precision=0.25157232704402516, recall=0.05102040816326531, fmeasure=0.08483563096500531)}

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rouge3','rougeL'])
scorer.score(df_hits.abstract.iloc[1],df_hits.abstract.iloc[24])

{'rouge1': Score(precision=0.39114391143911437, recall=0.13520408163265307, fmeasure=0.20094786729857822),
 'rouge2': Score(precision=0.06296296296296296, recall=0.021711366538952746, fmeasure=0.032288698955365625),
 'rouge3': Score(precision=0.0037174721189591076, recall=0.0012787723785166241, fmeasure=0.0019029495718363462),
 'rougeL': Score(precision=0.2177121771217712, recall=0.07525510204081633, fmeasure=0.1118483412322275)}

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rouge3','rougeL'])
scorer.score(df_hits.abstract.iloc[1],df_hi.abstract.iloc[2])

{'rouge1': Score(precision=0.23976608187134502, recall=0.18141592920353983, fmeasure=0.2065491183879093),
 'rouge2': Score(precision=0.011764705882352941, recall=0.008888888888888889, fmeasure=0.010126582278481011),
 'rouge3': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.12280701754385964, recall=0.09292035398230089, fmeasure=0.10579345088161209)}

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rouge3','rougeL'])
scorer.score(df_hits.abstract.iloc[1],df_his2.abstract.iloc[0])

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0),
 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0),
 'rouge3': Score(precision=1.0, recall=1.0, fmeasure=1.0),
 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rouge3','rougeL'])
scorer.score(df_hits.abstract.iloc[1],df_his2.abstract.iloc[1])

{'rouge1': Score(precision=0.310126582278481, recall=0.2168141592920354, fmeasure=0.25520833333333337),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rouge3': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.13924050632911392, recall=0.09734513274336283, fmeasure=0.11458333333333331)}

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rouge3','rougeL'])
scorer.score(df_hits.abstract.iloc[10],df_his2.abstract.iloc[10])

{'rouge1': Score(precision=0.22185430463576158, recall=0.29130434782608694, fmeasure=0.25187969924812026),
 'rouge2': Score(precision=0.026578073089700997, recall=0.034934497816593885, fmeasure=0.03018867924528302),
 'rouge3': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.12251655629139073, recall=0.1608695652173913, fmeasure=0.13909774436090228)}

In [ ]:
import pandas as pd

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd
import sys
sys.path.insert(0, "../")

In [ ]:
def replace_none(X):
    if X == '':
        X = np.nan
    return X

In [ ]:
def build_model(max_epochs, vec_size, alpha, tagged_data):
    
    model = Doc2Vec(vector_size=vec_size,
               alpha=alpha,
               min_alpha=0.00025,
               min_count=1,
               dm=1)
    
    model.build_vocab(tag_data)
    
    # With the model built we simply train on the data.
    
    for epoch in range(max_epochs):
        print(f"Iteration {epoch}")
        model.train(tag_data,
                   total_examples=model.corpus_count,
                   epochs=model.epochs)

        # Here I decrease the learning rate. 

        model.alpha -= 0.0002

        model.min_alpha = model.alpha
    
    # Now simply save the model to avoid training again. 
    
    model.save("/content/drive/MyDrive/COVID_MEDICAL_DOCS_w2v_MODEL.model")
    print("Model Saved")
    return model

In [ ]:
                                                                                                      !pip install transformers

     |████████████████████████████████| 3.8 MB 7.8 MB/s 
     |████████████████████████████████| 596 kB 52.3 MB/s 
     |████████████████████████████████| 895 kB 51.2 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 40.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
corona_df = pd.read_csv("/content/drive/MyDrive/kaggle_covid-19_open_csv_format.csv")

In [ ]:
corona_df.isnull().sum()

Unnamed: 0       0
doc_id           1
source           1
title          944
abstract      8783
text_body        1
dtype: int64

In [ ]:
corona_df['title'] = corona_df['title'].apply(replace_none)
corona_df['abstract'] = corona_df['abstract'].apply(replace_none)
corona_df = corona_df.dropna()

w2v_data_body = list(corona_df['abstract'])
w2v_data_title = list(corona_df['title'])

w2v_total_data = w2v_data_body + w2v_data_title

In [ ]:
tag_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(w2v_total_data)]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
model = build_model(max_epochs=5, vec_size=10, alpha=0.025, tagged_data=tag_data)

2022-03-22 06:43:07,241 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2022-03-22 06:43:07,244 : INFO : collecting all words and their counts
2022-03-22 06:43:07,246 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-03-22 06:43:07,648 : INFO : collected 58079 word types and 8576 unique tags from a corpus of 8576 examples and 1392279 words
2022-03-22 06:43:07,652 : INFO : Loading a fresh vocabulary
2022-03-22 06:43:08,007 : INFO : effective_min_count=1 retains 58079 unique words (100% of original 58079, drops 0)
2022-03-22 06:43:08,009 : INFO : effective_min_count=1 leaves 1392279 word corpus (100% of original 1392279, drops 0)
2022-03-22 06:43:08,187 : INFO : deleting the raw counts dictionary of 58079 items
2022-03-22 06:43:08,190 : INFO : sample=0.001 downsamples 32 most-common words
2022-03-22 06:43:08,193 : INFO : downsampling leaves estimated 1048361 word corpus (75.3% of prior 1392279)
2022-03-22 06:43:08,452 

Iteration 0


2022-03-22 06:43:22,076 : INFO : EPOCH 1 - PROGRESS: at 20.49% examples, 441784 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:23,078 : INFO : EPOCH 1 - PROGRESS: at 42.64% examples, 436417 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:23,946 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-22 06:43:23,984 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-22 06:43:24,008 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-22 06:43:24,010 : INFO : EPOCH - 1 : training on 1392279 raw words (1057393 effective words) took 2.9s, 359956 effective words/s
2022-03-22 06:43:25,045 : INFO : EPOCH 2 - PROGRESS: at 20.94% examples, 438953 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:26,070 : INFO : EPOCH 2 - PROGRESS: at 44.03% examples, 437293 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:26,915 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-22 06:43:26,944 : INFO : worker thread 

Iteration 1


2022-03-22 06:43:36,895 : INFO : EPOCH 1 - PROGRESS: at 21.35% examples, 446278 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:37,920 : INFO : EPOCH 1 - PROGRESS: at 35.77% examples, 359436 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:39,056 : INFO : EPOCH 1 - PROGRESS: at 54.80% examples, 314184 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:39,429 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-22 06:43:39,459 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-22 06:43:39,494 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-22 06:43:39,502 : INFO : EPOCH - 1 : training on 1392279 raw words (1057132 effective words) took 3.6s, 290875 effective words/s
2022-03-22 06:43:40,522 : INFO : EPOCH 2 - PROGRESS: at 12.22% examples, 280347 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:41,599 : INFO : EPOCH 2 - PROGRESS: at 22.26% examples, 227303 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:43:42,631 :

...

Iteration 3


2022-03-22 06:44:11,243 : INFO : EPOCH 1 - PROGRESS: at 20.49% examples, 441444 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:44:12,250 : INFO : EPOCH 1 - PROGRESS: at 42.64% examples, 435022 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:44:13,139 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-22 06:44:13,180 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-22 06:44:13,195 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-22 06:44:13,197 : INFO : EPOCH - 1 : training on 1392279 raw words (1056885 effective words) took 3.0s, 357183 effective words/s
2022-03-22 06:44:14,228 : INFO : EPOCH 2 - PROGRESS: at 20.90% examples, 440332 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:44:15,231 : INFO : EPOCH 2 - PROGRESS: at 43.56% examples, 439083 words/s, in_qsize 6, out_qsize 0
2022-03-22 06:44:16,105 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-22 06:44:16,167 : INFO : worker thread 

Iteration 4


2022-03-22 06:44:26,086 : INFO : EPOCH 1 - PROGRESS: at 20.49% examples, 441202 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:44:27,092 : INFO : EPOCH 1 - PROGRESS: at 42.64% examples, 435048 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:44:27,988 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-22 06:44:28,034 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-22 06:44:28,050 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-22 06:44:28,053 : INFO : EPOCH - 1 : training on 1392279 raw words (1056949 effective words) took 3.0s, 355475 effective words/s
2022-03-22 06:44:29,078 : INFO : EPOCH 2 - PROGRESS: at 20.94% examples, 444283 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:44:30,084 : INFO : EPOCH 2 - PROGRESS: at 44.02% examples, 444027 words/s, in_qsize 5, out_qsize 0
2022-03-22 06:44:30,885 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-22 06:44:30,920 : INFO : worker thread 

Model Saved


In [ ]:
model.wv.similar_by_word("detection")

2022-03-22 06:44:50,852 : INFO : precomputing L2-norms of word weight vectors


[('detecting', 0.9471670389175415),
 ('xtag', 0.9188621044158936),
 ('fi-nvct', 0.9184064269065857),
 ('filmarray', 0.9098459482192993),
 ('gpv', 0.9032878875732422),
 ('phe-cov', 0.9016375541687012),
 ('respifinder19h', 0.8948487639427185),
 ('cross-neutralization', 0.8896530866622925),
 ('uptakes', 0.8856788873672485),
 ('pcr-based', 0.8834467530250549)]

In [ ]:
model.wv.similar_by_word("diagnostic")

[('point-of-care', 0.9424715042114258),
 ('potentially-resistant', 0.9410459995269775),
 ('standardized', 0.9294978976249695),
 ('hints', 0.9240816831588745),
 ('syndromic', 0.9157186150550842),
 ('specially', 0.9102920293807983),
 ('workup', 0.9095381498336792),
 ('surrogate', 0.9086937308311462),
 ('virus-based', 0.908418595790863),
 ('respondent-driven', 0.9041361808776855)]

In [ ]:
voc_vec = word2vec.Word2Vec(vocab, min_count=1)

2022-03-22 06:41:39,132 : INFO : collecting all words and their counts
2022-03-22 06:41:39,139 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-22 06:41:39,147 : INFO : collected 15 word types from a corpus of 16 raw words and 2 sentences
2022-03-22 06:41:39,155 : INFO : Loading a fresh vocabulary
2022-03-22 06:41:39,159 : INFO : effective_min_count=1 retains 15 unique words (100% of original 15, drops 0)
2022-03-22 06:41:39,165 : INFO : effective_min_count=1 leaves 16 word corpus (100% of original 16, drops 0)
2022-03-22 06:41:39,167 : INFO : deleting the raw counts dictionary of 15 items
2022-03-22 06:41:39,172 : INFO : sample=0.001 downsamples 15 most-common words
2022-03-22 06:41:39,176 : INFO : downsampling leaves estimated 2 word corpus (13.7% of prior 16)
2022-03-22 06:41:39,180 : INFO : estimated required memory for 15 words and 100 dimensions: 19500 bytes
2022-03-22 06:41:39,182 : INFO : resetting layer weights
2022-03-22 06:41:39,200 : INFO :

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/metadata.csv')
df.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(482221, 19)

In [ ]:
title = df.copy()
title = title.dropna(subset=['title'])
title['title'] = title['title'].str.replace('[^a-zA-Z]', ' ', regex=True)
title['title'] = title['title'].str.lower()
title.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,clinical features of culture proven mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,nitric oxide a pro inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,surfactant protein d and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,role of endothelin in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [ ]:
import json
import os
datafiles = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if not (filename==''):
            if(filename in shaid):
                ifile = os.path.join(dirname, filename)
                if ifile.split(".")[-1] == "json":
                    datafiles.append(ifile)

In [ ]:
ArrBodyText = []
for file in datafiles:
    with open(file,'r')as f:
        doc = json.load(f)
    id = doc['paper_id'] 
    bodytext = ''
    for item in doc['body_text']:
        bodytext = bodytext + item['text']
        
    ArrBodyText.append({id:bodytext})

In [ ]:
included_vaccine = title.loc[title['keyword_vaccine'] != -1]
included_vaccine

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,keyword_vaccine
103,n9sih438,4994fa72322bbf19120592304d92629226948d8e,PMC,rapid identification of malaria vaccine candid...,10.1371/journal.pone.0000645,PMC1920550,17653272,cc-by,To identify malaria antigens for vaccine devel...,2007-07-25,"Villard, Viviane; Agak, George W.; Frank, Géra...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/4994fa72322bbf1912059...,document_parses/pmc_json/PMC1920550.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,32
106,1n69h3i3,bada3cc0577a226694617d77f95ac443c6431969,PMC,dna vaccines against protozoan parasites adva...,10.1155/2007/90520,PMC1940056,17710244,no-cc,"Over the past 15 years, DNA vaccines have gone...",2007-06-24,"Dumonteil, Eric",J Biomed Biotechnol,NaN,NaN,NaN,document_parses/pdf_json/bada3cc0577a226694617...,document_parses/pmc_json/PMC1940056.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,4
120,z2u5frvq,9992ffbec7c8256d057b05a858cde82d36d64ae3,PMC,antibody based hiv vaccines recent developm...,10.1371/journal.pmed.0040348,PMC2100141,18052607,cc-by,The authors discuss humoral immune responses t...,2007-12-01,"Montefiori, David; Sattentau, Quentin; Flores,...",PLoS Med,NaN,NaN,NaN,document_parses/pdf_json/9992ffbec7c8256d057b0...,document_parses/pmc_json/PMC2100141.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,21
154,8rrkf78o,235855da90347b259f4d6b244821f3c1c264c04e,PMC,expression of foot and mouth disease virus cap...,10.1371/journal.pone.0002273,PMC2386233,18509464,cc-by,BACKGROUND: Foot-and-mouth disease (FMD) is a ...,2008-05-28,"Li, Zhiyong; Yi, Yongzhu; Yin, Xiangping; Zhan...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/235855da90347b259f4d6...,document_parses/pmc_json/PMC2386233.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,134
190,pr9i9swk,dad72c9c3422ef1c9b9149e8dd567de494d46a8a,PMC,nasal delivery of an adenovirus based vaccine ...,10.1371/journal.pone.0003548,PMC2569416,18958172,cc-by,Pre-existing immunity to human adenovirus sero...,2008-10-29,"Croyle, Maria A.; Patel, Ami; Tran, Kaylie N.;...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/dad72c9c3422ef1c9b914...,document_parses/pmc_json/PMC2569416.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481970,cxcsblpg,35292b492d52cd4d91967a43338fb6ab4347352a,Medline; PMC,regulatory t cells as adjuvant target for enha...,10.1007/s13337-013-0187-3,PMC3889236,24426307.0,green-oa,CD4(+)CD25(+)FoxP3(+) regulatory T cells (Treg...,2013-11-29,"Bayry, Jagadeesh",VirusDisease,NaN,NaN,NaN,document_parses/pdf_json/35292b492d52cd4d91967...,document_parses/pmc_json/PMC3889236.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/24426307/;...,12790966.0,70
482034,jk04u17k,1b500daef057d370c3cb1d3953580ae2cc1bb10c,Elsevier; Medline; PMC,a novel liposome polymer hybrid nanoparticles ...,10.1016/j.nano.2020.102338,PMC7664365,33197626.0,no-cc,DNA vaccine is an attractive immune platform f...,2020-11-13,"Zhao, Zhangting; Ma, Xingyuan; Zhang, Ruihuan;...",Nanomedicine,NaN,NaN,NaN,document_parses/pdf_json/1b500daef057d370c3cb1...,document_parses/pmc_json/PMC7664365.xml.json,https://doi.org/10.1016/j.nano.2020.102338; ht...,226840603.0,94
482051,2ows227z,NaN,Medline; PMC,optimal allocation of the limited covid vac...,10.3390/jcm10040591,PMC7914460,33557344.0,cc-by,Initial supply of the coronavirus disease (COV...,2021-02-04,"Shim, Eunha",J Clin Med,NaN,NaN,NaN,NaN,document_parses/pmc_json/PMC7914460.xml.json,https://doi.org/10.3390/jcm10040591; https://w...,231873458.0,43
482078,ozf05l65,2a25a19235fe3811aa052fd57a6c3a7c6716abdf,Elsevier; Medline; PMC,preparing for pandemics rna vaccines at the f...,10.1016/j.ymthe.2020.06.017,PMC7309706,32579882.0,no-cc,NaN,2020-06-23,"Erasmus, Jesse H.; Fuller, Deborah Heydenburg",Mol Ther,N

In [ ]:
title['keyword_vaccine'] = title['title'].str.find('vaccine') 
title.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,keyword_vaccine
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,clinical features of culture proven mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,-1
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,nitric oxide a pro inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,-1
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,surfactant protein d and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,-1
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,role of endothelin in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,-1
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,-1


In [ ]:
included_vaccine = title.loc[title['keyword_vaccine'] != -1]
included_vaccine

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,keyword_vaccine
103,n9sih438,4994fa72322bbf19120592304d92629226948d8e,PMC,rapid identification of malaria vaccine candid...,10.1371/journal.pone.0000645,PMC1920550,17653272,cc-by,To identify malaria antigens for vaccine devel...,2007-07-25,"Villard, Viviane; Agak, George W.; Frank, Géra...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/4994fa72322bbf1912059...,document_parses/pmc_json/PMC1920550.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,32
106,1n69h3i3,bada3cc0577a226694617d77f95ac443c6431969,PMC,dna vaccines against protozoan parasites adva...,10.1155/2007/90520,PMC1940056,17710244,no-cc,"Over the past 15 years, DNA vaccines have gone...",2007-06-24,"Dumonteil, Eric",J Biomed Biotechnol,NaN,NaN,NaN,document_parses/pdf_json/bada3cc0577a226694617...,document_parses/pmc_json/PMC1940056.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,4
120,z2u5frvq,9992ffbec7c8256d057b05a858cde82d36d64ae3,PMC,antibody based hiv vaccines recent developm...,10.1371/journal.pmed.0040348,PMC2100141,18052607,cc-by,The authors discuss humoral immune responses t...,2007-12-01,"Montefiori, David; Sattentau, Quentin; Flores,...",PLoS Med,NaN,NaN,NaN,document_parses/pdf_json/9992ffbec7c8256d057b0...,document_parses/pmc_json/PMC2100141.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,21
154,8rrkf78o,235855da90347b259f4d6b244821f3c1c264c04e,PMC,expression of foot and mouth disease virus cap...,10.1371/journal.pone.0002273,PMC2386233,18509464,cc-by,BACKGROUND: Foot-and-mouth disease (FMD) is a ...,2008-05-28,"Li, Zhiyong; Yi, Yongzhu; Yin, Xiangping; Zhan...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/235855da90347b259f4d6...,document_parses/pmc_json/PMC2386233.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,134
190,pr9i9swk,dad72c9c3422ef1c9b9149e8dd567de494d46a8a,PMC,nasal delivery of an adenovirus based vaccine ...,10.1371/journal.pone.0003548,PMC2569416,18958172,cc-by,Pre-existing immunity to human adenovirus sero...,2008-10-29,"Croyle, Maria A.; Patel, Ami; Tran, Kaylie N.;...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/dad72c9c3422ef1c9b914...,document_parses/pmc_json/PMC2569416.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481970,cxcsblpg,35292b492d52cd4d91967a43338fb6ab4347352a,Medline; PMC,regulatory t cells as adjuvant target for enha...,10.1007/s13337-013-0187-3,PMC3889236,24426307.0,green-oa,CD4(+)CD25(+)FoxP3(+) regulatory T cells (Treg...,2013-11-29,"Bayry, Jagadeesh",VirusDisease,NaN,NaN,NaN,document_parses/pdf_json/35292b492d52cd4d91967...,document_parses/pmc_json/PMC3889236.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/24426307/;...,12790966.0,70
482034,jk04u17k,1b500daef057d370c3cb1d3953580ae2cc1bb10c,Elsevier; Medline; PMC,a novel liposome polymer hybrid nanoparticles ...,10.1016/j.nano.2020.102338,PMC7664365,33197626.0,no-cc,DNA vaccine is an attractive immune platform f...,2020-11-13,"Zhao, Zhangting; Ma, Xingyuan; Zhang, Ruihuan;...",Nanomedicine,NaN,NaN,NaN,document_parses/pdf_json/1b500daef057d370c3cb1...,document_parses/pmc_json/PMC7664365.xml.json,https://doi.org/10.1016/j.nano.2020.102338; ht...,226840603.0,94
482051,2ows227z,NaN,Medline; PMC,optimal allocation of the limited covid vac...,10.3390/jcm10040591,PMC7914460,33557344.0,cc-by,Initial supply of the coronavirus disease (COV...,2021-02-04,"Shim, Eunha",J Clin Med,NaN,NaN,NaN,NaN,document_parses/pmc_json/PMC7914460.xml.json,https://doi.org/10.3390/jcm10040591; https://w...,231873458.0,43
482078,ozf05l65,2a25a19235fe3811aa052fd57a6c3a7c6716abdf,Elsevier; Medline; PMC,preparing for pandemics rna vaccines at the f...,10.1016/j.ymthe.2020.06.017,PMC7309706,32579882.0,no-cc,NaN,2020-06-23,"Erasmus, Jesse H.; Fuller, Deborah Heydenburg",Mol Ther,N

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
!pip install kneed

In [ ]:

!pip install transformers==2.6.0

import transformers
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel, DistilBertModel, DistilBertTokenizer, BertTokenizer, BertForTokenClassification
import numpy as np

import nltk
nltk.download('punkt')
from nltk import sent_tokenize
%tensorflow_version 1.x
from keras.preprocessing.sequence import pad_sequences

from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import json
import matplotlib.pyplot as plt
import timeit
import torch
import textwrap
wrapper = textwrap.TextWrapper(width=70)
SEED = 1234
torch.manual_seed(SEED)

     |████████████████████████████████| 540 kB 11.9 MB/s 
     |████████████████████████████████| 895 kB 48.6 MB/s 
     |████████████████████████████████| 1.2 MB 43.8 MB/s 
     |████████████████████████████████| 132 kB 58.4 MB/s 
     |████████████████████████████████| 5.6 MB 58.2 MB/s 
     |████████████████████████████████| 8.6 MB 56.4 MB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 138 kB 63.1 MB/s 
     |████████████████████████████████| 127 kB 61.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
[nltk_data] Downloading package punkt to /root/nltk_da

In [ ]:

# Distil version is fine for this task
BERT_pretrained_weights = 'distilbert-base-uncased' }

# for token classification we used 
BERTforTokenClassification_config_directory = 'BERT_dir' 
token_label_files = 'BERT_dir/POS2idx.json'

GPT2_config_directory = 'GPT2_dir' 
print('which BERT pre-trained ? ',BERT_pretrained_weights)
print('where is BERT token classifier dir ? ',BERTforTokenClassification_config_directory)
print('where is GPT2 dir ? ',GPT2_config_directory)

which BERT pre-trained ?  distilbert-base-uncased
where is BERT token classifier dir ?  BERT_dir
where is GPT2 dir ?  GPT2_dir


In [ ]:
print('----loading pre-trained BERT----')
BERT_pretrained = DistilBertModel.from_pretrained(
                  BERT_pretrained_weights)
tokenizer_pretrained = DistilBertTokenizer.from_pretrained(
                  BERT_pretrained_weights)
print('----loading token labels----')

----loading pre-trained BERT----
----loading token labels----


In [ ]:
use_GPU_BERT_pre_trained = False 
use_GPU_BERT_token_classifier = False 
use_GPU_GPT_generator = True
if torch.cuda.is_available():
  print('cuda is available')
  device = 'cuda'
  print('device is set to cuda')
if not torch.cuda.is_available():
  print('cuda is not available')
  device = 'cpu'
  print('device is set to cpu')
  use_GPU_BERT_pre_trained = False
  use_GPU_BERT_token_classifier = False
  use_GPU_GPT_generator = False

print(' ')
print('use GPU for pre-trained BERT?' ,use_GPU_BERT_pre_trained)
print('use GPU for BERT token classifier ?' ,use_GPU_BERT_token_classifier)
print('use GPU for GPT2?' ,use_GPU_GPT_generator)

cuda is not available
device is set to cpu
 
use GPU for pre-trained BERT? False
use GPU for BERT token classifier ? False
use GPU for GPT2? False


In [ ]:
#@title Main text file

#@markdown indicate the text file to be summarized
use_input_text = False

input_file = 'directory/file.txt' 
max_len = 50 #@param {type:"integer",max:512}

#@markdown or copy paste your input here and check the box
use_input_text = True #@param {type:"boolean"}
input_text = "'Two months after it was firstly reported, the novel coronavirus disease COVID-19 has already spread worldwide. However, the vast majority of reported infections have occurred in China. To assess the effect of early travel restrictions adopted by the health authorities in China, we have implemented an epidemic metapopulation model that is fed with mobility data corresponding to 2019 and 2020. This allows to compare two radically different scenarios, one with no travel restrictions and another in which mobility is reduced by a travel ban. Our findings indicate that i) travel restrictions are an effective measure in the short term, however, ii) they are ineffective when it comes to completely eliminate the disease. The latter is due to the impossibility of removing the risk of seeding the disease to other regions. Our study also highlights the importance of developing more realistic models of behavioral changes when a disease outbreak is unfolding.'" #@param {type:"string"}

if not use_input_text:
  # open the txt file that is included
  with open(input_file, 'r') as file:
    input_text = file.read().replace('\n', '')

# split text to sentences
paragraph_split = sent_tokenize(input_text)

print('input text has',len(paragraph_split) ,'sentences.')

print('tokenizing sentences')

input_tokens = []
for i in paragraph_split:
  input_tokens.append(tokenizer_pretrained.encode(i, 
                              add_special_tokens=True))
temp = []
for i in input_tokens:
  temp.append(len(i))
if np.max(temp) > max_len:
  raise ValueError('sentence longer than the max_len')
if np.max(temp) > 512:
  print('warning: sentence longer than 512')
  print('suggest to change max_len to 512, the remainder will be truncated')
input_ids = pad_sequences(input_tokens, 
                          maxlen=max_len, dtype="long", 
                          value=0, 
                          truncating="post", 
                          padding="post")

print('creating attention masks')

attention_masks = []
for sent in input_ids:
  att_mask = [int(token_id > 0) for token_id in sent]  # create a list of 0 and 1.
  attention_masks.append(att_mask)  # basically attention_masks is a list of list

input_ids = torch.tensor(input_ids)  
attention_mask = torch.tensor(attention_masks)

input text has 7 sentences.
tokenizing sentences
creating attention masks


In [ ]:
input_ids

tensor([[  101,  1005,  2048,  ...,     0,     0,     0],
        [  101,  2174,  1010,  ...,     0,     0,     0],
        [  101,  2000, 14358,  ...,     0,     0,     0],
        ...,
        [  101,  2256,  9556,  ...,     0,     0,     0],
        [  101,  1996,  3732,  ...,     0,     0,     0],
        [  101,  2256,  2817,  ...,     0,     0,     0]])

In [ ]:
number_extract = 6 
if use_GPU_BERT_pre_trained:
  input_ids = input_ids.to(device)
  BERT_pretrained = BERT_pretrained.to(device)
  attention_mask = attention_mask.to(device)

if not use_GPU_BERT_pre_trained:
  input_ids = input_ids.to('cpu')
  BERT_pretrained = BERT_pretrained.to('cpu')
  attention_mask = attention_mask.to('cpu')

with torch.no_grad():
  last_hidden_states = BERT_pretrained(input_ids, 
                             attention_mask=attention_mask)

sentence_features = last_hidden_states[0][:,0,:].detach().cpu().numpy()

print('performing k-medoid clustering with '
        ,number_extract,' clusters')

kmeans = KMeans(n_clusters=number_extract, 
                random_state=0).fit(sentence_features)
cluster_center = kmeans.cluster_centers_
nbrs = NearestNeighbors(n_neighbors= 1, 
                        algorithm='brute').fit(sentence_features)
distances, indices = nbrs.kneighbors(
                  cluster_center.reshape(number_extract,-1))

indices = np.sort(indices.reshape(1,-1))
topic_answer = []

for i in indices[0]:
  topic_answer.append(paragraph_split[i])

print('result:')

print('the ',number_extract,' extracted sentences are')
for i in topic_answer:
  print(i)

topic_answer_string = ''
for topic in topic_answer:
  topic_answer_string = topic_answer_string + ' '+ topic

performing k-medoid clustering with  6  clusters
result:
the  6  extracted sentences are
'Two months after it was firstly reported, the novel coronavirus disease COVID-19 has already spread worldwide.
However, the vast majority of reported infections have occurred in China.
To assess the effect of early travel restrictions adopted by the health authorities in China, we have implemented an epidemic metapopulation model that is fed with mobility data corresponding to 2019 and 2020.
This allows to compare two radically different scenarios, one with no travel restrictions and another in which mobility is reduced by a travel ban.
Our findings indicate that i) travel restrictions are an effective measure in the short term, however, ii) they are ineffective when it comes to completely eliminate the disease.
The latter is due to the impossibility of removing the risk of seeding the disease to other regions.


In [ ]:
distances, indices = nbrs.kneighbors(
                  cluster_center.reshape(number_extract,-1))
indices 

array([[4],
       [5],
       [3],
       [1],
       [2],
       [0]])

In [ ]:
print (topic_answer_string)

 'Two months after it was firstly reported, the novel coronavirus disease COVID-19 has already spread worldwide. However, the vast majority of reported infections have occurred in China. To assess the effect of early travel restrictions adopted by the health authorities in China, we have implemented an epidemic metapopulation model that is fed with mobility data corresponding to 2019 and 2020. This allows to compare two radically different scenarios, one with no travel restrictions and another in which mobility is reduced by a travel ban. Our findings indicate that i) travel restrictions are an effective measure in the short term, however, ii) they are ineffective when it comes to completely eliminate the disease. The latter is due to the impossibility of removing the risk of seeding the disease to other regions.


In [ ]:
wrapper.wrap(input_text)

["'Two months after it was firstly reported, the novel coronavirus",
 'disease COVID-19 has already spread worldwide. However, the vast',
 'majority of reported infections have occurred in China. To assess the',
 'effect of early travel restrictions adopted by the health authorities',
 'in China, we have implemented an epidemic metapopulation model that is',
 'fed with mobility data corresponding to 2019 and 2020. This allows to',
 'compare two radically different scenarios, one with no travel',
 'restrictions and another in which mobility is reduced by a travel ban.',
 'Our findings indicate that i) travel restrictions are an effective',
 'measure in the short term, however, ii) they are ineffective when it',
 'comes to completely eliminate the disease. The latter is due to the',
 'impossibility of removing the risk of seeding the disease to other',
 'regions. Our study also highlights the importance of developing more',
 'realistic models of behavioral changes when a disease outbreak

In [ ]:
list_keywords_str2 = 'Pathogenesis of Virus-Induced DemyelinationPublisher Summary Demyelination is component diseases humans are sclerosing panencephalitis SSPE leukoencephalopathy PML are number virus infections animals involve demyelination serve models demyelinating diseases diseases viruses have be demyelination situations demyelinating disease chapter reviews architecture organization CNS considers is interaction viruses CNS cells discusses immunology CNS differs aspects rest body models demyelination have Viruses disease have features include RNA viruses viruses chapter attempts summarize factors demyelination their features mechanisms'
title = 'A data-driven assessment of early travel restrictions related to the spreading of the novel COVID-19 within mainland China '

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 8.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=cae37e5fe8d83cad034466f5a8e3e82468b4a196fc4273f8ad5a4d933b2eb63a
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
import numpy as np
import pandas as pd
import scipy as sc

import os
import json
import warnings

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
question_embedding = model.encode(['What do we know about virus genetics, origin, and evolution?'])

queries = ['What is known about transmission, incubation, and environmental stability?', 'What do we know about COVID-19 risk factors?', 
           'What do we know about virus genetics, origin, and evolution?', 'What do we know about vaccines and therapeutics?',
           'Are there geographic variations in the rate of COVID-19 spread?', 'Are there geographic variations in the mortality rate of COVID-19?',
           'Is there any evidence to suggest geographic based virus mutations?','What do we know about diagnostics and surveillance?',
           'What do we know about non-pharmaceutical interventions?','What has been published about medical care?',
           'What has been published about ethical and social science considerations?', 'What has been published about information sharing and inter-sectoral collaboration?']

query_embeddings = model.encode(queries)

In [ ]:
count = 0
file_exts = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        count += 1
        file_ext = filename.split(".")[-1]
        file_exts.append(file_ext)

file_ext_set = set(file_exts)
file_ext_list = list(file_ext_set)

count = 0
for root, folders, filenames in os.walk('/kaggle/input'):
    print(root, folders)
    
json_folder_path = "/kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset"
json_file_name = os.listdir(json_folder_path)[0]
json_path = os.path.join(json_folder_path, json_file_name)

with open(json_path) as json_file:
    json_data = json.load(json_file)
    
json_data_df = pd.io.json.json_normalize(json_data)

FileNotFoundError: ignored